## Loading dataset

Here we have three text files in hindi:
1. hindi_1.txt - around 180MB in size

2. hindi_2.txt - around 45MB in size

3. hindi_3.txt - around 9.3GB in size - this is too large. Hence we will only pick the first 1Lakh lines from this file for creating our vocabulary set

In [5]:
with open('hindi_1.txt', 'r', encoding='utf-8') as f:
    text1 = f.read()
with open('hindi_2.txt', 'r', encoding='utf-8') as f:
    text2 = f.read()

In [6]:
import fileinput
N = 100000
line_counter = 0
text3 = ''
for line in fileinput.input("hindi_3.txt", encoding="utf-8"):
    line_counter += 1
    text3 += line + '\n'
    if line_counter == N:
        break
print(f'Added {N} lines to text3')

Added 100000 lines to text3


In [7]:
text1[:100]

'नर्मदा, जिसे रेवा के नाम से भी जाना जाता है, मध्य भारत की एक नदी और भारतीय उपमहाद्वीप की पांचवीं सबस'

In [8]:
text2[:100]

'निर्देशांक: 27°30′N 79°24′E / 27.5°N 79.4°E / 27.5; 79.4\nनिनौरा संकलपुर फर्रुखाबाद, फर्रुखाबाद, उत्त'

In [9]:
text3[:100]

'शारदा पारा के मिलन चैiक से आज महापौर देवेन्द्र यादव, पार्षद छोटे लाल चैधरी के साथ वार्ड का भ्रमण करत'

In [10]:
print("length of dataset1 in characters: ", len(text1))
print("length of dataset2 in characters: ", len(text2))
print("length of dataset3 in characters: ", len(text3))

length of dataset1 in characters:  71184826
length of dataset2 in characters:  16976318
length of dataset3 in characters:  32497056


In [34]:
text = text1 + text2 + text3

In [35]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  120658200


In [36]:
text[:1000]

'नर्मदा, जिसे रेवा के नाम से भी जाना जाता है, मध्य भारत की एक नदी और भारतीय उपमहाद्वीप की पांचवीं सबसे लंबी नदी है। यह गोदावरी नदी और कृष्णा नदी के बाद भारत के अंदर बहने वाली तीसरी सबसे लंबी नदी है। मध्य प्रदेश राज्य में इसके विशाल योगदान के कारण इसे "मध्य प्रदेश की जीवन रेखा" भी कहा जाता है। यह उत्तर और दक्षिण भारत के बीच एक पारंपरिक सीमा की तरह कार्य करती है। यह अपने उद्गम से पश्चिम की ओर 1,312 किमी चल कर खंभात की खाड़ी, अरब सागर में जा मिलती है।\nनर्मदा, मध्य भारत के मध्य प्रदेश और गुजरात राज्य में बहने वाली एक प्रमुख नदी है। मैकल पर्वत के अमरकण्टक शिखर से नर्मदा नदी की उत्पत्ति हुई है। इसकी लम्बाई प्रायः 1312 किलोमीटर है। यह नदी पश्चिम की तरफ जाकर खम्बात की खाड़ी में गिरती है।\nनर्मदा नदी का उद्गम मध्यप्रदेश के अनूपपुर जिले में विंध्याचल और सतपुड़ा पर्वतश्रेणियों के पूर्वी संधिस्थल पर स्थित अमरकंटक में नर्मदा कुंड से हुआ है। नदी पश्चिम की ओर सोनमुद से बहती हुई, एक चट्टान से नीचे गिरती हुई कपिलधारा नाम की एक जलप्रपात बनाती है। घुमावदार मार्ग और प्रबल वेग के साथ घने जंगलो और चट्टानों

## Vocabulary size

In [16]:
vocab = sorted(list(set(text)))
print('Length of Vocabulary:',len(vocab))

Length of Vocabulary: 10001


In [17]:
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))
print("length of text:", len(text))
print("length of tokens:", len(tokens))

length of text: 120658200
length of tokens: 304795096


In [18]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [19]:
stats = get_stats(tokens)

## Top 5 pairs with highest frequency

In [20]:
list(stats.items())[:5]

[((224, 164), 71087117),
 ((164, 168), 3760226),
 ((168, 224), 2835629),
 ((164, 176), 6114494),
 ((176, 224), 4266312)]

In [21]:
top_pair = max(stats, key=stats.get)
top_pair

(224, 164)

In [22]:
chr(224), chr(164)

('à', '¤')

Now we will merge the repetitive pairs and replace with 256 + {index of pair}

In [23]:
def merge(ids, pair, idx):
  i = 0
  newids = []
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  
  return newids

In [25]:
num_merges = len(vocab) - 256
num_merges

9745

## Compression Ratio

In [41]:
ids = list(tokens)
merges = {}
for i in range(30):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  ids = merge(ids, pair, idx)
  merges[pair] = idx

print('Merges completed')

Merges completed


In [43]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 304795096
ids length: 55698259
compression ratio: 5.47X


In [44]:
with open('merges.pkl', 'wb') as file:
    pickle.dump(merges, file)
with open('vocab.pkl', 'wb') as file:
    pickle.dump(vocab, file)

## Using regex

In [46]:
import string, re

tokenizer_hindi_pattern=re.compile(r'(['+string.punctuation+r'\u0964\u0965\uAAF1\uAAF0\uABEB\uABEC\uABED\uABEE\uABEF\u1C7E\u1C7F'+r'])')
pattern_num_seq=re.compile(r'([0-9]+ [,.:/] )+[0-9]+')

def tokenize_hindi_text(text): 
    """tokenize string for Indian language scripts using Brahmi-derived scripts

    A trivial tokenizer which just tokenizes on the punctuation boundaries. 
    This also includes punctuations for the Indian language scripts (the 
    purna virama and the deergha virama). This is a language independent 
    tokenizer

    Args:
        text (str): text to tokenize

    Returns:
        list: list of tokens

    """
    tok_str=tokenizer_hindi_pattern.sub(r' \1 ',text.replace('\t',' '))
    s=re.sub(r'[ ]+',' ',tok_str).strip(' ')
    
    # do not tokenize numbers and dates
    new_s=''
    prev=0
    for m in pattern_num_seq.finditer(s):
        start=m.start()
        end=m.end()
        if start>prev:
            new_s=new_s+s[prev:start]
            new_s=new_s+s[start:end].replace(' ','')
            prev=end
   
    new_s=new_s+s[prev:]
    s=new_s
    
    return s.split(' ')

In [47]:
tokenize_hindi_text('नर्मदा, जिसे रेवा के नाम से भी जाना जाता है, मध्य भारत की एक नदी और भारतीय उपमहाद्वीप की पांचवीं सबस')

['नर्मदा',
 ',',
 'जिसे',
 'रेवा',
 'के',
 'नाम',
 'से',
 'भी',
 'जाना',
 'जाता',
 'है',
 ',',
 'मध्य',
 'भारत',
 'की',
 'एक',
 'नदी',
 'और',
 'भारतीय',
 'उपमहाद्वीप',
 'की',
 'पांचवीं',
 'सबस']